# Import related pacakages

In [1]:
pip install wget
pip install goatools

# Download data

In [ ]:
from goatools.obo_parser import GODag
import wget

go_fn = wget.download('http://geneontology.org/ontology/go-basic.obo')
go = GODag(go_fn, optional_attrs=['relationship'])

In [1]:
from goatools.rpt.rpt_lev_depth import RptLevDepth
from goatools.obo_parser import GODag
obodag = GODag("./data/go-basic.obo")
rptobj = RptLevDepth(obodag)
rptobj.write_summary_cnts_all()

./data/go-basic.obo: fmt(1.2) rel(2020-10-09) 47,313 GO Terms
Dep <-Depth Counts->  <-Level Counts->
Lev   BP    MF    CC    BP    MF    CC
--- ----  ----  ----  ----  ----  ----
00     1     1     1     1     1     1
01    28    19     5    28    19     5
02   240   128   926   404   151   928
03  1176   578   681  2217   896   779
04  2327  1551   889  4824  2021  1041
05  3623  4808   674  6854  5061   644
06  4537  1888   474  6890  2012   436
07  4713  1106   310  4729   750   228
08  4286   538   158  2052   197   107
09  3509   307    59   634    47    25
10  2217   158    18   215     6     1
11  1288    85     1    42    17     1
12   655    11     0     0     0     0
13   228     0     0     0     0     0
14    49     0     0     0     0     0
15     9     0     0     0     0     0
16     4     0     0     0     0     0


In [5]:

#GO_level_ano
import pandas as pd
GO_level_space_out=pd.DataFrame()
i=0
for eachgo in obodag:
    eg_term = obodag[eachgo]  
    GO_level_space_out.loc[i,'GO']=eachgo
    GO_level_space_out.loc[i,'name']=eg_term.name   
    GO_level_space_out.loc[i,'level']=eg_term.level
    GO_level_space_out.loc[i,'namespace']=eg_term.namespace   
    i=i+1
GO_level_space_out.to_csv('./analysis/GO_level_space_out.csv', header=True, index=False)

In [9]:
#ec2go
#http://current.geneontology.org/ontology/external2go/ec2go

import re
import pandas as pd
import numpy as np

outFile=open('./analysis/GO_ec.csv','w')
outFile.write('GO,ec\n')
for eachdata in open('./data/ec2go.txt'):
    if re.search('!',eachdata):
        pass
    else:
        data=eachdata.split(' > ')
        ec=data[0].split('EC:')[1]
        GO=data[1].split('; ')[1].split('\n')[0]
        spec=ec.split('.')
        if len(spec)>3:
            outstr=GO+','+ec+'\n'
            outFile.write(outstr)
outFile.close()
GO_ec=pd.read_csv('./analysis/GO_ec.csv')#
print('uniEC数目:'+str(len(np.unique(GO_ec['ec']))))
print('uniGO数目:'+str(len(np.unique(GO_ec['GO']))))

uniEC数目:4264
uniGO数目:4488


In [17]:
GO_ec = pd.read_csv('./analysis/GO_ec.csv')
ec_max_df = pd.read_csv('./analysis/ec_max_df.csv', index_col=0)
GO_ec_kcat = pd.DataFrame()
for index, row in GO_ec.iterrows():
    if row['ec'] in ec_max_df.index:
        GO_ec_kcat.loc[index,'GO'] = row['GO']
        GO_ec_kcat.loc[index,'ec'] = row['ec']
        GO_ec_kcat.loc[index,'kcat'] = ec_max_df.loc[row['ec'],'kcat']
GO_ec_kcat.to_csv('./analysis/GO_ec_kcat.csv', header=True, index=False)
GO_ec_kcat.head(5)

,GO,ec,kcat
0,GO:0004022,1.1.1.1,4808.0
3,GO:0004316,1.1.1.100,344.7
4,GO:0102131,1.1.1.100,344.7
5,GO:0102132,1.1.1.100,344.7
9,GO:0008743,1.1.1.103,551.7


In [18]:
GO_level_space_out = pd.read_csv('./analysis/GO_level_space_out.csv', index_col=0)
GO_level_space_out.head(5)

,name,level,namespace
GO,,,
GO:0000001,mitochondrion inheritance,6.0,biological_process
GO:0000002,mitochondrial genome maintenance,6.0,biological_process
GO:0000003,reproduction,1.0,biological_process
GO:0000006,high-affinity zinc transmembrane transporter a...,7.0,molecular_function
GO:0000007,low-affinity zinc ion transmembrane transporte...,7.0,molecular_function


In [33]:
import numpy as np

ec='1.1.1.100'
GO_ec_kcat = pd.read_csv('./analysis/GO_ec_kcat.csv')
GO_list = list(GO_ec_kcat[GO_ec_kcat['ec']==ec]['GO'])
print(GO_list)
same_level_GO_df = pd.DataFrame()
for eachgo in GO_list:
    go_level=GO_level_space_out.loc[eachgo,'level']
    go_type=GO_level_space_out.loc[eachgo,'namespace']
    tmp1=GO_level_space_out[GO_level_space_out['namespace']==go_type]
    tmp2=tmp1[tmp1['level']==go_level]
    kcat_list=[]
    for eachgo2 in tmp2.index:
        if eachgo2 in list(GO_ec_kcat['GO']):
            kcat_list.append(np.max(GO_ec_kcat[GO_ec_kcat['GO']==eachgo2]['kcat']))
    if len(kcat_list)>1:
        same_level_GO_df.loc[eachgo,'kcat']=np.max(kcat_list)
np.max(same_level_GO_df['kcat'])

['GO:0004316', 'GO:0102131', 'GO:0102132']


57500000.0

In [4]:
import numpy as np
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *
import re

json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enz_model=get_enzyme_constraint_model(json_model_path)



b1858
b1859
b1857


In [13]:
gene_GO = pd.read_csv('./data/gene_go.csv')
GO_level_space_out = pd.read_csv('./analysis/GO_level_space_out.csv', index_col=0)
GO_ec_kcat = pd.read_csv('./analysis/GO_ec_kcat.csv')

In [17]:
reaction = enz_model.reactions.get_by_id('ACGAptspp')
gpr_list = reaction.gene_reaction_rule.split(' ')
gene_list=[]
gene_kcat_list=[]
for eachdata in gpr_list:
    if len(eachdata)>4:
        if eachdata not in gene_list:
            gene_list.append(eachdata)
            GO_list=list(gene_GO[gene_GO['gene']==eachdata]['GO'])
            same_level_GO_df = pd.DataFrame()
            for eachgo in GO_list:
                if eachgo in GO_level_space_out.index:
                    go_level=GO_level_space_out.loc[eachgo,'level']
                    go_type=GO_level_space_out.loc[eachgo,'namespace']
                    tmp1=GO_level_space_out[GO_level_space_out['namespace']==go_type]
                    tmp2=tmp1[tmp1['level']==go_level]
                    kcat_list=[]
                    for eachgo2 in tmp2.index:
                        if eachgo2 in list(GO_ec_kcat['GO']):
                            kcat_list.append(np.max(GO_ec_kcat[GO_ec_kcat['GO']==eachgo2]['kcat']))
                    if len(kcat_list)>1:
                        same_level_GO_df.loc[eachgo,'kcat']=np.max(kcat_list)
            gene_kcat_list.append(np.max(same_level_GO_df['kcat'])) 

     
print(np.max(gene_kcat_list))

57500000.0


In [ ]:
same_level_GO_df = pd.DataFrame()
for eachgo in GO_list:
    go_level=GO_level_space_out.loc[eachgo,'level']
    go_type=GO_level_space_out.loc[eachgo,'namespace']
    tmp1=GO_level_space_out[GO_level_space_out['namespace']==go_type]
    tmp2=tmp1[tmp1['level']==go_level]
    kcat_list=[]
    for eachgo2 in tmp2.index:
        if eachgo2 in list(GO_ec_kcat['GO']):
            kcat_list.append(np.max(GO_ec_kcat[GO_ec_kcat['GO']==eachgo2]['kcat']))
    if len(kcat_list)>1:
        same_level_GO_df.loc[eachgo,'kcat']=np.max(kcat_list)
np.max(same_level_GO_df['kcat'])